In [1]:
import nibabel as nib
import numpy as np
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.padding import PKCS7
import os
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import logging
import pyspark
from nibabel.filebasedimages import FileBasedImage
import io
import pandas as pd
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
CT_SCANS = "../rawdata/ct_scans"
INFECTION_MASKS = "../rawdata/infection_mask"
LUNGANDINFECTION_MASKS = "../rawdata/lung_and_infection_mask"
LUNG_MASKS = "../rawdata/lung_mask"
OUTPUT_DIR = "../rawdata/"

In [3]:
ct_nii: FileBasedImage = nib.load(f"{CT_SCANS}/coronacases_org_001.nii")
header = ct_nii.header
affine = ct_nii.affine
data = ct_nii.get_fdata()
data = np.array(data)
data

array([[[-998., -998., -999., ..., -998., -998., -998.],
        [-998., -998., -999., ..., -998., -998., -998.],
        [-998., -998., -999., ..., -998., -998., -998.],
        ...,
        [-998., -999., -998., ..., -998., -998., -998.],
        [-998., -999., -998., ..., -998., -998., -998.],
        [-998., -999., -998., ..., -998., -998., -998.]],

       [[-998., -998., -999., ..., -998., -998., -998.],
        [-998., -998., -999., ..., -998., -998., -998.],
        [-998., -998., -999., ..., -998., -998., -998.],
        ...,
        [-998., -999., -998., ..., -998., -998., -998.],
        [-998., -999., -998., ..., -998., -998., -998.],
        [-998., -999., -998., ..., -998., -998., -998.]],

       [[-998., -998., -999., ..., -998., -998., -998.],
        [-998., -998., -999., ..., -998., -998., -998.],
        [-998., -998., -999., ..., -998., -998., -998.],
        ...,
        [-998., -999., -998., ..., -998., -998., -998.],
        [-998., -999., -998., ..., -998., -99

In [4]:
spark = SparkSession.builder \
    .appName("NIfTI_Processing") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.memory", "20g") \
    .config("spark.driver.maxResultSize", "20g") \
    .getOrCreate()
sc = spark.sparkContext

24/12/07 19:34:10 WARN Utils: Your hostname, SerVer resolves to a loopback address: 127.0.1.1; using 192.168.1.250 instead (on interface enp3s0f0)
24/12/07 19:34:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/07 19:34:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load into RDDs

In [ ]:
def extract_metadata():
    # Extract metadata from the downloaded dataset
    path = os.path.join(OUTPUT_DIR, "metadata.csv")
    metadata = pd.read_csv(path)
    logger.info("Metadata extracted successfully!")
    metadata.replace("../input/covid19-ct-scans/", "", regex=True, inplace=True)
    return metadata

def extract_image_data(metadata: pd.DataFrame):
    # Convert DataFrame to np array
    image_paths: np.ndarray = metadata.to_numpy().flatten()
    # Extract image data
    def load_image(x):
        path = os.path.join(OUTPUT_DIR, x)
        nifti_data = nib.load(path)
        header = nifti_data.header
        affine = nifti_data.affine
        image_data = nifti_data.get_fdata()
        image_data = np.array(image_data)

        return image_data, header, affine, x

    image_rdds = sc.parallelize(image_paths).map(load_image)

    return image_rdds

def save_image_data(image_rdds: pyspark.rdd.PipelinedRDD):
    # Save image data to disk
    def save_image(x):
        image_data, header, affine, file_name = x
        nifti_data = nib.Nifti1Image(image_data, affine, header)
        os.makedirs("output", exist_ok=True)
        folder = file_name.split("/")[0]
        os.makedirs(f"output/{folder}", exist_ok=True)
        nifti_data.to_filename(f"output/{file_name}")

    image_rdds.foreach(save_image)

In [ ]:
metadata = extract_metadata()
rdds = extract_image_data(metadata)
save_image_data(rdds)

INFO:__main__:Metadata extracted successfully!
(630, 630, 39)                                                    (0 + 72) / 72]
(630, 630, 45)
(630, 630, 39)
(630, 630, 45)
(630, 630, 45)
(630, 630, 42)
24/12/07 19:38:05 ERROR Executor: Exception in task 65.0 in stage 2.0 (TID 209)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
          

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 65 in stage 2.0 failed 1 times, most recent failure: Lost task 65.0 in stage 2.0 (TID 209) (192.168.1.250 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, it

erator))
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^
  [Previous line repeated 1 more time]
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/rdd.py", line 840, in func
    return f(iterator)
           ^^^^^^^^^^^
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/rdd.py", line 1763, in processPartition
    f(x)
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-p

24/12/07 19:38:07 WARN TaskSetManager: Lost task 47.0 in stage 2.0 (TID 191) (192.168.1.250 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 65 in stage 2.0 failed 1 times, most recent failure: Lost task 65.0 in stage 2.0 (TID 209) (192.168.1.250 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/aphuc/web/data_science/pytorch-notebook/ETL-MI/.venv/lib/python3.12/site-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, it

## Encrypted Data

In [ ]:
def encrypt_nifti(file_path, password, output_path):
    # Read the NIfTI image
    nifti_img = nib.load(file_path)
    data = nifti_img.get_fdata()
    header = nifti_img.header
    
    # Serialize the data to bytes
    data_bytes = data.tobytes()

    # Derive a key from the password
    salt = os.urandom(16)  # Generate a random salt
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())  # Derive the key

    # Encrypt the data
    iv = os.urandom(16)  # Initialization vector
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    
    # Padding for block cipher
    padder = PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(data_bytes) + padder.finalize()

    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    # Save the encrypted data and metadata
    with open(output_path, 'wb') as f:
        f.write(salt)  # Write salt for key derivation
        f.write(iv)    # Write IV for decryption
        f.write(encrypted_data)  # Write encrypted data

    print(f"Encrypted NIfTI image saved to {output_path}")

In [ ]:
encrypt_nifti(f"{CT_SCANS}/coronacases_org_001.nii", "258456C@p", "encrypted.nii")

Encrypted NIfTI image saved to encrypted.nii


In [ ]:
def decrypt_nifti(encrypted_file_path, password, output_path):
    # Read the encrypted file
    with open(encrypted_file_path, 'rb') as f:
        salt = f.read(16)  # Read salt
        iv = f.read(16)    # Read IV
        encrypted_data = f.read()  # Read encrypted data

    # Derive the key from the password
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())

    # Decrypt the data
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Remove padding
    unpadder = PKCS7(algorithms.AES.block_size).unpadder()
    data_bytes = unpadder.update(padded_data) + unpadder.finalize()

    # Deserialize data
    nifti_img = nib.load(f"{CT_SCANS}/coronacases_org_001.nii")
    original_shape = nifti_img.shape
    data = np.frombuffer(data_bytes, dtype=np.float64).reshape(original_shape)

    # Save the decrypted NIfTI file
    decrypted_nifti = nib.Nifti1Image(data, nifti_img.affine, nifti_img.header)
    nib.save(decrypted_nifti, output_path)

    print(f"Decrypted NIfTI image saved to {output_path}")

In [ ]:
decrypt_nifti("encrypted.nii", "258456C@p", "decrypted.nii")

FileNotFoundError: No such file or no access: '{CT_SCANS}/coronacases_org_001.nii'